In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

import itertools
import time
import sys  

sys.path.insert(0, '../')

from soundings.deep_learning import mlutilities as ml
from soundings.deep_learning import tf_neuralnetwork as nn
from soundings.deep_learning import tf_metrics as metrics

from soundings.preprocessing import dataloader as dl

from soundings.plotting import radiosonde_plotting, results
from soundings.utils import radiosonde_products, radiosonde_utils

from IPython.display import display
from ipywidgets import FloatProgress

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

In [2]:
processed_vol = '/mnt/data1/stock/mlsoundings/processed/'
PLOT_CMAP = 'RdYlBu_r'

gpus = tf.config.get_visible_devices('GPU')
for device in gpus:
    print(device)
    # tf.config.experimental.set_visible_devices(device, 'GPU')
    tf.config.experimental.set_memory_growth(device, True)
print()
!nvidia-smi

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')

Mon Nov  2 16:28:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:3D:00.0 Off |                  Off |
| 33%   30C    P8    12W / 260W |     16MiB / 24212MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----

In [3]:
f = '/mnt/data1/stock/mlsoundings/preprocessed.npz'
# np.savez(f, raob=raob, rap=rap,
#          goes=goes, rtma=rtma, sonde_files=sonde_files)

container = np.load(f)

print(container.files)

raob = container['raob']
rap = container['rap']
goes = container['goes']
rtma = container['rtma']
sonde_files = container['sonde_files']

print(raob.shape, rap.shape, goes.shape, rtma.shape)

['raob', 'rap', 'goes', 'rtma', 'sonde_files']
(81276, 256, 4) (81276, 256, 4) (81276, 3, 3, 8) (81276, 3, 3, 3)


In [4]:
diff = raob[:,:,dl.TEMPERATURE] - rap[:,:,dl.TEMPERATURE]

threshold = 20
inds = np.unique(np.where(abs(diff) >= threshold)[0])

raob = np.delete(raob, inds, 0)
rap  = np.delete(rap, inds, 0)
goes = np.delete(goes, inds, 0)
rtma = np.delete(rtma, inds, 0)
sonde_files = np.delete(sonde_files, inds, 0)

In [5]:
Xtrain, Ttrain, Xtest, Ttest = ml.partition(rap, raob, (0.8,0.2), shuffle=True)

input_dims = [dl.PRESSURE, dl.TEMPERATURE, dl.DEWPOINT, dl.ALTITUDE]

Xtrain = Xtrain[:, :, input_dims]
Ttrain = Ttrain[:, :, [dl.TEMPERATURE, dl.DEWPOINT]]

Xtest = Xtest[:, :, input_dims]
Ttest = Ttest[:, :, [dl.TEMPERATURE, dl.DEWPOINT]]
Xtrain.shape, Ttrain.shape, Xtest.shape, Ttest.shape

((64947, 256, 4), (64947, 256, 2), (16237, 256, 4), (16237, 256, 2))

In [ ]:
nnet = nn.SkipNeuralNetwork(Xtrain.shape[1:], [256, 512, 1024], 
                            [(5, 1), (5, 1), (5, 1)],
                            Ttrain.shape[1:])
# nnet.model.summary()
nnet.train(Xtrain, Ttrain, 30, 128, method='adam', verbose=True,
           learning_rate=0.0001, validation=[Xtest, Ttest])

epoch: 0, rmse 4.27874, val_rmse 3.52006
epoch: 6, rmse 3.42483, val_rmse 3.36804
epoch: 12, rmse 3.36903, val_rmse 3.31149
epoch: 18, rmse 3.33695, val_rmse 3.29303


In [ ]:
results.plot_loss(nnet)

In [ ]:
Y = nnet.use(Xtest)

In [ ]:
# 1.055, 0.995 (8,1)
results.plot_altitude_rmse_verticle(None, None, Ttest[:,:,0],
                                    Xtest[:,:,dl.TEMPERATURE], Y=Y[:,:,0])

In [ ]:
# 4.549, 3.590 (8,1)
results.plot_altitude_rmse_verticle(None, None, Ttest[:,:,1],
                                    Xtest[:,:,dl.DEWPOINT], Y=Y[:,:,1])

In [ ]:
i = 7
sounding = dict()
sounding[radiosonde_utils.PRESSURE_COLUMN_KEY] = Xtest[i,:,dl.PRESSURE]
sounding[radiosonde_utils.TEMPERATURE_COLUMN_KEY] = Ttest[i,:,0]
sounding[radiosonde_utils.NWP_TEMPERATURE_COLUMN_KEY] = Xtest[i,:,dl.TEMPERATURE]
sounding[radiosonde_utils.PREDICTED_TEMPERATURE_COLUMN_KEY] = Y[i,:,0]

radiosonde_plotting.plot_nwp_ml_sounding(sounding);

In [ ]:
sounding = dict()
sounding[radiosonde_utils.PRESSURE_COLUMN_KEY] = Xtest[i,:,dl.PRESSURE]
sounding[radiosonde_utils.TEMPERATURE_COLUMN_KEY] = Ttest[i,:,1]
sounding[radiosonde_utils.NWP_TEMPERATURE_COLUMN_KEY] = Xtest[i,:,dl.DEWPOINT]
sounding[radiosonde_utils.PREDICTED_TEMPERATURE_COLUMN_KEY] = Y[i,:,1]

radiosonde_plotting.plot_nwp_ml_sounding(sounding);

In [ ]:
import metpy

In [ ]:
i = 0
pressure = Xtrain[i, :, dl.PRESSURE]
temperature = Xtrain[i, :, dl.TEMPERATURE]
dewpoint = Xtrain[i, :, dl.DEWPOINT]
metpy.calc.surface_based_cape_cin(pressure, temperature, dewpoint)